# Analyse de sentiments pour Air Paradis
**Auteur:** Didier DRACHE  
**Date:** Mars-Avril 2025

Ce notebook présente le développement d'un système d'analyse de sentiments pour détecter les bad buzz potentiels sur les réseaux sociaux pour la compagnie aérienne Air Paradis.

## 1. Introduction et Configuration

### 1.1 Importation des bibliothèques nécessaires

Les bibliothèques ci-dessous sont utilisées pour l'analyse de données, la modélisation et le suivi des expériences.

In [ ]:
# 1. Introduction et Configuration
# Projet: Détection de sentiment pour Air Paradis
# Auteur: Didier DRACHE
# Date de début: 22 mars 2025

# 1.1 Importation des bibliothèques nécessaires
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # Ou '2' pour cacher aussi les warnings
import sys
import time
import json
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Tuple, Optional, Union, Any

# Statistiques et métriques
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_curve, auc
)

# Prétraitement et modélisation
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier

# Deep Learning
import tensorflow as tf

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    Dense, Embedding, LSTM, GRU, Bidirectional, Conv1D, GlobalMaxPooling1D,
    Dropout, BatchNormalization, Input, MaxPooling1D
)
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam

# Tracking des expérimentations
import mlflow
import mlflow.sklearn
import mlflow.keras

# Import des fonctions utilitaires personnalisées
# Ce fichier contient toutes les fonctions réutilisables
from config import CONFIG, update_sample_size, update_lemmatization
try:
    from utils import *
except ImportError:
    print("Fichier utils.py introuvable, veuillez exécuter d'abord la cellule de création des utilitaires")
    
# Configuration de visualisation
plt.style.use('seaborn-v0_8-darkgrid')
sns.set(style="darkgrid")

# Configuration pour éviter les avertissements
import warnings
warnings.filterwarnings('ignore')


# Forcer l'utilisation du GPU 1 (second GPU)
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# Vérifier que TensorFlow voit le GPU correctement
print("TensorFlow GPUs disponibles:", tf.config.list_physical_devices('GPU'))

# Configurer l'allocation mémoire dynamique pour éviter de saturer la VRAM
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Permettre une allocation mémoire dynamique (libérable)
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("Allocation mémoire dynamique activée pour tous les GPUs")
    except RuntimeError as e:
        print("Erreur lors de la configuration des GPUs:", e)

# Libérer explicitement la mémoire CUDA
def clear_gpu_memory():
    import gc
    gc.collect()
    tf.keras.backend.clear_session()
    print("Mémoire GPU libérée")

download_nltk_resources()

### 1.2 Création de la structure de répertoires

Configuration de TensorFlow pour utiliser efficacement les GPUs disponibles (RTX 3090) avec allocation mémoire dynamique.

In [ ]:
# 1.2 Création de la structure de répertoires
created_dirs = create_directory_structure()
print(f"Répertoires créés: {created_dirs}")

### 1.3 Configuration de MLflow

In [ ]:
# 1.3 Configuration de MLflow
experiment_id = setup_mlflow_experiment("Air Paradis - Analyse de Sentiment")
print(f"MLflow experiment ID: {experiment_id}")

### 1.4 Informations sur le projet et contexte métier

- **Client:** Air Paradis (compagnie aérienne)
- **Objectif:** Développer un modèle de prédiction du sentiment associé aux tweets
- **Contexte:** Anticiper les bad buzz sur les réseaux sociaux
- **Approches à tester:**
  1. Modèle sur mesure simple (classique)
  2. Modèle sur mesure avancé (deep learning)
  3. Modèle avancé BERT
  4. Modèle DistilBERT (alternative légère à BERT)

### 1.5 Informations sur l'environnement d'exécution

In [ ]:
# 1.5 Informations sur l'environnement d'exécution

from IPython.display import Markdown, display

# Collecte des informations sur l'environnement
import platform, re
system_info = f"**Système:** {platform.system()} {platform.release()}"
python_info = f"**Python:** {platform.python_version()}"

# Informations GPU
gpu_info = []
if tf.config.list_physical_devices('GPU'):
    gpus = tf.config.list_physical_devices('GPU')
    for gpu in gpus:
        gpu_info.append(f"GPU disponible: {gpu.name}")
        
        # Informations sur la mémoire GPU
        try:
            import torch
            gpu_info.append(f"Mémoire GPU (CUDA): {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
        except:
            gpu_info.append("Impossible d'obtenir les informations de mémoire GPU via PyTorch")
else:
    gpu_info.append("Aucun GPU détecté, l'exécution sera plus lente sur CPU")

# Construction du markdown
markdown_content = f"""
- {system_info}
- {python_info}
"""

# Ajouter les informations GPU
for info in gpu_info:
    info_md =info.replace('/physical_device:','')
    info_md = re.sub(r'^([^:]+):', r'**\1:**', info_md)
    markdown_content += f"- {info_md}\n"

# Afficher le markdown
print_md(markdown_content)

### 1.6 Paramètres globaux

In [ ]:
# 1.6 Paramètres globaux

# Paramètres globaux pour le projet

# taille_echantillon = 100000
# taille_echantillon = 10000 # pour debug
# taille_echantillon = 20000 # pour debug
# taille_echantillon = None # tout le jeu de donnés
taille_echantillon = 100000
Utiliser_Lemmatisation = False

# Mettre à jour la configuration avec la taille d'échantillon
CONFIG = update_sample_size(taille_echantillon)
CONFIG = update_lemmatization(Utiliser_Lemmatisation)

# Fixer les graines aléatoires pour la reproductibilité
np.random.seed(CONFIG["RANDOM_SEED"])
tf.random.set_seed(CONFIG["RANDOM_SEED"])

display(CONFIG)

- **Graine aléatoire:** 42
- **Taille de test:** 20%
- **Taille de validation:** 10%
- **Vocabulaire maximum:** 50,000 mots
- **Taille de batch:** 64
- **Époques:** 10
- **Longueur max des séquences:** 100
- **Dimension des embeddings:** 100

## 2. Exploration et Préparation des Données

Cette section couvre le chargement, l'exploration et la préparation des données Sentiment140 pour l'entraînement des modèles.

### 2.1 Chargement des données

In [ ]:
# 2. Exploration et Préparation des Données
# 2.1 Chargement des données

# Ces modèles sont maintenant définis dans utils.py pour faciliter leur réutilisation
from utils import load_sentiment140_data

# Chargement des données
result = load_sentiment140_data(CONFIG["DATA_PATH"])
df = result[0]  # On garde seulement le DataFrame, pas le temps d'exécution

### 2.2 Analyse exploratoire des données

Examinons la structure des données, leur distribution et leurs caractéristiques principales pour mieux comprendre le corpus de tweets.

In [ ]:
# 2.2 Analyse exploratoire des données

stats_markdown = f"""
#### Statistiques générales du dataset:
- **Nombre total d'observations:** {df.shape[0]:,}
- **Nombre de variables:** {df.shape[1]}"""
print_md(stats_markdown)

# print("\nAperçu des données:")
print_md(f"""#### Aperçu des données:""")
display(df.head())

# print("\nInformations sur les variables:")
print_md(f"""\n#### Informations sur les variables:""")
display(df.info())

# print("\nStatistiques descriptives:")
print_md(f"""\n#### Statistiques descriptives du dataset:""")
display(df.describe(include='all'))

# Distribution des sentiments
print_md(f"""\n#### Distribution des sentiments dans le corpus:""")
stats_markdown = f"""
- **Distribution des sentiments:** 
  - Négatif: {df[df['sentiment'] == 0].shape[0]:,} tweets ({df[df['sentiment'] == 0].shape[0]/df.shape[0]*100:.1f}%)
  - Positif: {df[df['sentiment'] == 1].shape[0]:,} tweets ({df[df['sentiment'] == 1].shape[0]/df.shape[0]*100:.1f}%)
"""
print_md(stats_markdown)

# print("\nDistribution des sentiments:")
sentiment_counts = df['sentiment'].value_counts().reset_index()
sentiment_counts.columns = ['Sentiment', 'Count']
sentiment_counts['Sentiment'] = sentiment_counts['Sentiment'].replace({0: 'Négatif', 1: 'Positif'})
display(sentiment_counts)

plt.figure(figsize=(10, 6))
sns.countplot(x='sentiment', hue='sentiment', data=df, palette=['red', 'green'],legend=False)
plt.title('Distribution des sentiments', fontsize=14)
plt.xlabel('Sentiment (0 = Négatif, 1 = Positif)')
plt.ylabel('Nombre de tweets')
plt.xticks([0, 1], ['Négatif', 'Positif'])
plt.savefig('visualisations/2_2_sentiment_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

### 2.3 Analyse du texte des tweets

Cette section explore la longueur des tweets et leur composition pour identifier des motifs potentiellement utiles pour la modélisation.

In [ ]:
# 2.3 Analyse du texte des tweets
print("\n=== Analyse du texte des tweets ===")

# Longueur des tweets
df['text_length'] = df['text'].apply(len)

text_stats_markdown = f"""
#### Caractéristiques des tweets:
- **Longueur des tweets:**
  - Moyenne: {df['text_length'].mean():.2f} caractères
  - Médiane: {df['text_length'].median():.2f} caractères
  - Minimum: {df['text_length'].min()} caractères
  - Maximum: {df['text_length'].max()} caractères
"""
print_md(text_stats_markdown)

plt.figure(figsize=(12, 6))
sns.histplot(data=df, x='text_length', hue='sentiment', 
             bins=50, kde=True, palette=['red', 'green'])
plt.title('Distribution de la longueur des tweets par sentiment', fontsize=14)
plt.xlabel('Longueur du tweet (caractères)')
plt.ylabel('Nombre de tweets')
plt.savefig('visualisations/2_3_tweet_length_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

# Nombre de mots par tweet
df['word_count'] = df['text'].apply(lambda x: len(str(x).split()))

text_stats_markdown = f"""
#### Nombre de mots:
- Moyenne: {df['word_count'].mean():.2f} mots
- Médiane: {df['word_count'].median():.2f} mots
- Minimum: {df['word_count'].min()} mots
- Maximum: {df['word_count'].max()} mots
"""
print_md(text_stats_markdown)

plt.figure(figsize=(12, 6))
sns.histplot(data=df, x='word_count', hue='sentiment', 
             bins=50, kde=True, palette=['red', 'green'])
plt.title('Distribution du nombre de mots par tweet', fontsize=14)
plt.xlabel('Nombre de mots')
plt.ylabel('Nombre de tweets')
plt.savefig('visualisations/2_3_word_count_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

### 2.4 Analyse des termes les plus fréquents

Identification des mots qui apparaissent le plus fréquemment dans les tweets positifs et négatifs pour comprendre les différences lexicales.

In [ ]:
# 2.4 Analyse des termes les plus fréquents
from collections import Counter
import re
import nltk
from nltk.corpus import stopwords

# Ces modèles sont maintenant définis dans utils.py pour faciliter leur réutilisation
from utils import get_top_words

# Télécharger les stopwords si nécessaire
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

# Créer un markdown pour résumer les termes fréquents
terms_markdown = "#### Termes les plus fréquents:\n\n"
terms_markdown += "| Rang | Terme négatif | Fréquence | Terme positif | Fréquence |\n"
terms_markdown += "|------|--------------|-----------|---------------|-----------|\n"

# Mots les plus fréquents dans les tweets négatifs
negative_tweets = df[df['sentiment'] == 0]['text']
negative_top_words = get_top_words(negative_tweets)

# Mots les plus fréquents dans les tweets positifs
positive_tweets = df[df['sentiment'] == 1]['text']
positive_top_words = get_top_words(positive_tweets)

for i in range(min(20, len(negative_top_words), len(positive_top_words))):
    neg_word, neg_count = negative_top_words[i]
    pos_word, pos_count = positive_top_words[i]
    terms_markdown += f"| {i+1} | {neg_word} | {neg_count:,} | {pos_word} | {pos_count:,} |\n"

print_md(terms_markdown)

# Visualisation des mots les plus fréquents
plt.figure(figsize=(20, 8))

# Tweets négatifs
plt.subplot(1, 2, 1)
words, counts = zip(*negative_top_words)
sns.barplot(x=list(counts), y=list(words), hue=list(words), palette='Reds_r',legend=False)
plt.title('Top 20 des mots - Tweets Négatifs', fontsize=14)
plt.xlabel('Nombre d\'occurrences')

# Tweets positifs
plt.subplot(1, 2, 2)
words, counts = zip(*positive_top_words)
sns.barplot(x=list(counts), y=list(words), hue=list(words), palette='Greens_r',legend=False)
plt.title('Top 20 des mots - Tweets Positifs', fontsize=14)
plt.xlabel('Nombre d\'occurrences')

plt.tight_layout()
plt.savefig('visualisations/2_4_top_words_by_sentiment.png', dpi=300, bbox_inches='tight')
plt.show()

### 2.5 Prétraitement des données

Nettoyage des textes et préparation des ensembles d'entraînement et de test pour la modélisation.

In [ ]:
# 2.5 Prétraitement des données

from joblib import Parallel, delayed
import multiprocessing

from utils import clean_text, preprocess_data, save_processed_data, load_processed_data

# Prétraitement des données

# Utiliser les paramètres de la configuration globale
result = preprocess_data(df, 
                        sample_size=CONFIG["SAMPLE_SIZE"], 
                        lemmatize=CONFIG["USE_LEMMATIZATION"])

X_train, X_test, y_train, y_test = result[0]  # On garde seulement les données, pas le temps d'exécution

save_processed_data(X_train, X_test, y_train, y_test)

print_md(f"""
#### Résultats du prétraitement:
- **Textes nettoyés** avec suppression des URLs, mentions, et caractères spéciaux
- **Lemmatisation: {"Appliquée" if CONFIG["USE_LEMMATIZATION"] else "Non appliquée"}**
- **Division des données:**
  - Ensemble d'entraînement: {len(X_train):,} exemples
  - Ensemble de test: {len(X_test):,} exemples
- **Distribution préservée** dans les ensembles d'entraînement et de test
- **Temps de prétraitement:** {result[1]:.2f} secondes
""")


## 3. Modèles Classiques ("Modèle sur mesure simple")

Cette section présente le développement et l'évaluation de modèles traditionnels de machine learning pour la classification de sentiment des tweets.

### 3.1 Préparation des fonctions pour les modèles classiques

Définition des fonctions d'entraînement, d'évaluation et de visualisation pour les modèles classiques.

In [ ]:
# 3. Modèles Classiques ("Modèle sur mesure simple")

# 3.1 Préparation des fonctions pour les modèles classiques

from utils import train_evaluate_classical_model

### 3.2 Entraînement et évaluation des modèles classiques

Implémentation et comparaison de plusieurs algorithmes de classification: Régression Logistique, Naive Bayes, SVM et Random Forest.

In [ ]:
# 3.2 Entraînement et évaluation des modèles classiques

from utils import train_all_classical_models

### 3.3 Exécution et visualisation des modèles classiques

Entraînement des modèles et visualisation comparative de leurs performances et temps d'exécution.

In [ ]:
# 3.3 Exécution et visualisation des modèles classiques

from utils import train_evaluate_classical_model, train_all_classical_models, print_md

# from utils import train_evaluate_classical_model, train_all_classical_models, print_md
# from visualization_utils import plot_model_comparison, plot_model_time_comparison
# import pandas as pd
# import os

# Charger les données prétraitées
X_train, X_test, y_train, y_test = load_processed_data()

# Entraîner tous les modèles classiques
print_md("#### ====== Entraînement des modèles classiques ======")

classical_results = train_all_classical_models(X_train, X_test, y_train, y_test)
print(f"Forme du DataFrame classical_results: {classical_results.shape}")

# Afficher les résultats
print_md(f"""#### ====== Résultats des modèles classiques ======""")

display(classical_results)

print_md(f"""
#### ====== Comparaison des performances des modèles classiques ======

| Modèle | Accuracy | F1 Score | Temps (s) | Taille (MB) |
|--------|----------|----------|-----------|-------------|
| {classical_results.iloc[0]['Modèle']} | {classical_results.iloc[0]['Accuracy']:.4f} | {classical_results.iloc[0]['F1 Score']:.4f} | {classical_results.iloc[0]['Temps (s)']:.2f} | {classical_results.iloc[0]['Taille (MB)']:.2f} |
| {classical_results.iloc[1]['Modèle']} | {classical_results.iloc[1]['Accuracy']:.4f} | {classical_results.iloc[1]['F1 Score']:.4f} | {classical_results.iloc[1]['Temps (s)']:.2f} | {classical_results.iloc[1]['Taille (MB)']:.2f} |
| {classical_results.iloc[2]['Modèle']} | {classical_results.iloc[2]['Accuracy']:.4f} | {classical_results.iloc[2]['F1 Score']:.4f} | {classical_results.iloc[2]['Temps (s)']:.2f} | {classical_results.iloc[2]['Taille (MB)']:.2f} |
| {classical_results.iloc[3]['Modèle']} | {classical_results.iloc[3]['Accuracy']:.4f} | {classical_results.iloc[3]['F1 Score']:.4f} | {classical_results.iloc[3]['Temps (s)']:.2f} | {classical_results.iloc[3]['Taille (MB)']:.2f} |
""")

# Visualiser les performances des modèles
plot_model_comparison(classical_results, 
                      metrics=['Accuracy', 'Precision', 'Recall', 'F1 Score'],
                      filename='visualisations/3_3_classical_models_comparison.png')
plot_model_comparison(classical_results, 
                      metrics=['Accuracy', 'Precision', 'Recall', 'F1 Score'])

# Visualiser les temps d'entraînement
plot_model_time_comparison(classical_results, 
                          filename='visualisations/3_3_classical_models_time_comparison.png')
plot_model_time_comparison(classical_results)

### 3.4 Analyse approfondie du meilleur modèle classique

Étude détaillée des features importantes et du comportement du modèle le plus performant.

In [ ]:
# 3.4 Analyse approfondie du meilleur modèle classique
# Trouver le meilleur modèle selon le F1 Score
best_model_idx = classical_results['F1 Score'].idxmax()
best_model_name = classical_results.loc[best_model_idx, 'Modèle']
print(f"\nMeilleur modèle classique: {best_model_name}")

# Charger le meilleur modèle
best_model_path = f"models/classical/{best_model_name.replace(' ', '_').lower()}.pkl"
with open(best_model_path, 'rb') as f:
    best_pipeline = pickle.load(f)

# Accéder au vectoriseur et au modèle
vectorizer = best_pipeline.named_steps['vectorizer']
model = best_pipeline.named_steps['model']

features_md = f"""
#### Features les plus importantes pour {best_model_name}

| Feature | Importance |
|---------|------------|
"""

# Analyse des features importantes (pour les modèles qui le supportent)
if hasattr(model, 'coef_'):
    feature_names = vectorizer.get_feature_names_out()
    
    # Pour les modèles avec une seule classe de sortie (ex: LogisticRegression avec deux classes)
    if len(model.coef_.shape) == 2 and model.coef_.shape[0] == 1:
        coefficients = model.coef_[0]
    else:
        coefficients = model.coef_
    
    # Créer un DataFrame avec les coefficients
    feature_importance = pd.DataFrame({
        'feature': feature_names,
        'importance': np.abs(coefficients)
    })
    
    # Trier par importance décroissante
    feature_importance = feature_importance.sort_values('importance', ascending=False)

    for i in range(min(10, len(feature_importance))):
        row = feature_importance.iloc[i]
        feat = row.get('feature')  # Utilisation de get() pour éviter les erreurs
        imp = row.get('importance')
        features_md += f"| {feat} | {imp:.4f} |\n"
    print_md(features_md)
    
    # Afficher les 20 features les plus importantes
    print("\nTop 20 des features les plus importantes:")
    display(feature_importance.head(20))

    # Visualiser les 20 features les plus importantes
    plt.figure(figsize=(12, 8))
    sns.barplot(x='importance', y='feature', data=feature_importance.head(20))
    plt.title(f'Top 20 des features les plus importantes - {best_model_name}', fontsize=14)
    plt.xlabel('Importance (valeur absolue du coefficient)')
    plt.ylabel('Feature')
    plt.tight_layout()
    plt.savefig(f'visualisations/3_4_feature_importance_{best_model_name.replace(" ", "_").lower()}.png', 
                dpi=300, bbox_inches='tight')
    plt.show()
elif hasattr(model, 'feature_importances_'):
    # Pour les modèles comme Random Forest
    feature_names = vectorizer.get_feature_names_out()
    feature_importance = pd.DataFrame({
        'feature': feature_names,
        'importance': model.feature_importances_
    })
    
    # Trier par importance décroissante
    feature_importance = feature_importance.sort_values('importance', ascending=False)
    
    # Afficher les 20 features les plus importantes
    print("\nTop 20 des features les plus importantes:")
    display(feature_importance.head(20))

    
    # Visualiser les 20 features les plus importantes
    plt.figure(figsize=(12, 8))
    sns.barplot(x='importance', y='feature', data=feature_importance.head(20))
    plt.title(f'Top 20 des features les plus importantes - {best_model_name}', fontsize=14)
    plt.xlabel('Importance de la feature')
    plt.ylabel('Feature')
    plt.tight_layout()
    plt.savefig(f'visualisations/3_4_feature_importance_{best_model_name.replace(" ", "_").lower()}.png', 
                dpi=300, bbox_inches='tight')
    plt.show()
    
else:
    print(f"L'extraction des features importantes n'est pas supportée pour le modèle {best_model_name}")

### 3.5 Échantillons de prédictions

Tests du modèle sur des exemples concrets de tweets pour évaluer qualitativement ses performances.

In [ ]:
# 3.5 Échantillons de prédictions
# Faire des prédictions sur quelques exemples
sample_texts = [
    "I absolutely love this product! It's amazing!",
    "This is the worst experience I've ever had.",
    "The flight was delayed by 2 hours and no compensation was offered.",
    "Air Paradis has the best customer service I've experienced.",
    "Not sure how I feel about this, it's just okay I guess."
]

# Nettoyer les textes
sample_cleaned = [clean_text(text) for text in sample_texts]

# Faire les prédictions
sample_predictions = best_pipeline.predict(sample_cleaned)
if hasattr(best_pipeline, 'predict_proba'):
    sample_proba = best_pipeline.predict_proba(sample_cleaned)[:, 1]
else:
    sample_proba = None

predictions_md = f"""
#### Exemples de prédictions avec {best_model_name}

| Texte | Prédiction | Confiance |
|-------|------------|-----------|
"""

for i, (text, pred) in enumerate(zip(sample_texts, sample_predictions)):
    sentiment = "Positif" if pred == 1 else "Négatif"
    confidence = sample_proba[i] if sample_proba is not None else "N/A"
    predictions_md += f"| {text} | {sentiment} | {confidence:.2f} |\n"

print_md(predictions_md)

### 3.6 Conclusion sur les modèles classiques

In [ ]:
# 3.6 Conclusion sur les modèles classiques

print_md( f"""
Le meilleur modèle classique est **{best_model_name}** avec:
- F1 Score: **{classical_results.loc[best_model_idx, 'F1 Score']:.4f}**
- Temps d'entraînement: **{classical_results.loc[best_model_idx, 'Temps (s)']:.2f} secondes**
- Taille du modèle: **{classical_results.loc[best_model_idx, 'Taille (MB)']:.2f} MB**

**Avantages des modèles classiques:**
- Rapides à entraîner
- Facilement interprétables
- Taille réduite facilitant le déploiement

**Limites des modèles classiques:**
- Ne capturent pas la sémantique complexe des textes
- Performances limitées par rapport aux approches avancées
- Ne prennent pas en compte l'ordre des mots et le contexte
""" )

## 4. Modèles Deep Learning ("Modèle sur mesure avancé")

Dans cette section, nous allons développer des modèles de deep learning plus sophistiqués pour l'analyse de sentiment des tweets. Ces modèles ont la capacité de capturer des relations complexes dans le texte grâce à différentes architectures neuronales.

### 4.1 Préparation des données pour les modèles deep learning

Les modèles de deep learning nécessitent une préparation spécifique des données textuelles, notamment la conversion en séquences numériques et le padding pour obtenir des longueurs uniformes.

In [ ]:
# 4. Modèles Deep Learning ("Modèle sur mesure avancé")
# 4.1 Préparation des données pour les modèles deep learning

from utils import prepare_data_for_dl

### 4.2 Chargement des embeddings pré-entrainés

Pour représenter efficacement les mots, nous utiliserons des embeddings pré-entraînés GloVe qui capturent la sémantique des mots dans un espace vectoriel. Ces embeddings peuvent améliorer significativement les performances des modèles de deep learning.

In [ ]:
# 4.2 Chargement des embeddings pré-entrainés

from utils import load_glove_embeddings, create_random_embeddings


### 4.3 Fonctions pour les modèles de deep learning

Nous définissons ici les fonctions nécessaires pour l'entraînement et l'évaluation des modèles deep learning, incluant les callbacks personnalisés et les métriques d'évaluation.

### 4.4 Définition des architectures de modèles deep learning

Nous implémentons deux architectures principales :
- **CNN (Convolutional Neural Networks)** : Efficaces pour capturer les motifs locaux dans le texte
- **LSTM (Long Short-Term Memory)** : Spécialisés dans la modélisation des dépendances séquentielles à long terme

Chaque architecture sera testée avec deux types d'embeddings : entraînables et pré-entraînés (GloVe).

In [ ]:
# 4.3 Fonctions pour les modèles de deep learning

from utils import get_callbacks, train_evaluate_dl_model

In [ ]:
# 4.4 Définition des architectures de modèles deep learning

# Ces modèles sont maintenant définis dans utils.py pour faciliter leur réutilisation
from utils import create_cnn_model, create_lstm_model

"""
Cette section utilise deux architectures principales:
- CNN (Convolutional Neural Networks) pour capturer les motifs locaux dans le texte
- LSTM (Long Short-Term Memory) pour modéliser les dépendances séquentielles à long terme

Chaque architecture est testée avec deux types d'embeddings: entraînables et pré-entraînés (GloVe).
"""

### 4.5 Entraînement et évaluation des modèles deep learning

Cette fonction orchestre l'entraînement de tous les modèles deep learning définis précédemment, avec différentes configurations d'embeddings. Nous collectons systématiquement les métriques de performance, les temps d'exécution et les tailles des modèles pour une comparaison objective.

In [ ]:
# 4.5 Entraînement et évaluation des modèles deep learning

# Ces modèles sont maintenant définis dans utils.py pour faciliter leur réutilisation
from utils import apply_augmentation
from utils import train_all_dl_models



### 4.6 Exécution et visualisation des modèles deep learning

Les graphiques ci-dessous présentent une comparaison des performances (F1 Score, Accuracy, etc.) et des ressources requises (temps d'entraînement, taille des modèles) pour les différentes architectures testées.

In [ ]:
# 4.6 Exécution et visualisation des modèles deep learning

# Charger les données prétraitées
print_md(f"#### ====== Entraînement des modèles deep learning ======")
X_train, X_test, y_train, y_test = load_processed_data()

# Entraîner tous les modèles deep learning
dl_results, tokenizer, word_index, vocab_size = train_all_dl_models(X_train, X_test, y_train, y_test)

# Afficher les résultats
# print("\n=== Résultats des modèles deep learning ===")
print_md(f"#### ====== Résultats des modèles deep learning ======")
display(dl_results)

# Visualiser les performances des modèles
plot_model_comparison(dl_results, 
                      metrics=['Accuracy', 'Precision', 'Recall', 'F1 Score'],
                      filename='visualisations/4_6_dl_models_comparison.png')
plot_model_comparison(dl_results, 
                      metrics=['Accuracy', 'Precision', 'Recall', 'F1 Score'])

# Visualiser les temps d'entraînement
plot_model_time_comparison(dl_results, 
                           filename='visualisations/4_6_dl_models_time_comparison.png')
plot_model_time_comparison(dl_results)

### 4.7 Analyse approfondie du meilleur modèle deep learning

Nous examinons en détail le modèle qui a obtenu les meilleures performances parmi les architectures deep learning testées.

In [ ]:
# 4.7 Analyse approfondie du meilleur modèle deep learning
# Trouver le meilleur modèle selon le F1 Score
best_model_idx = dl_results['F1 Score'].idxmax()
best_model_name = dl_results.loc[best_model_idx, 'Modèle']
print_md(f"\n#### Meilleur modèle deep learning: **{best_model_name}**")

# Charger le meilleur modèle
best_model_path = f"models/deeplearning/{best_model_name.replace(' ', '_').lower()}.keras"
best_model = tf.keras.models.load_model(best_model_path)

### 4.8 Échantillons de prédictions avec le meilleur modèle

Évaluons qualitativement notre meilleur modèle deep learning sur quelques exemples concrets de tweets.

In [ ]:
# 4.8 Échantillons de prédictions avec le meilleur modèle
# Exemples de textes
sample_texts = [
    "I absolutely love this airline! Best flight ever!",
    "This is the worst airline experience I've ever had.",
    "The flight was delayed by 2 hours and no compensation was offered.",
    "Air Paradis has the best customer service I've experienced.",
    "Not sure how I feel about this flight, it's just okay I guess."
]

# Préparer les textes pour la prédiction
sample_cleaned = [clean_text(text) for text in sample_texts]
sample_sequences = tokenizer.texts_to_sequences(sample_cleaned)
sample_padded = pad_sequences(sample_sequences, maxlen=CONFIG["MAX_SEQUENCE_LENGTH"])

# Faire les prédictions
sample_proba = best_model.predict(sample_padded)
sample_predictions = (sample_proba > 0.5).astype(int)

# Afficher les résultats
print("\nExemples de prédictions avec le meilleur modèle deep learning:")
for i, (text, pred) in enumerate(zip(sample_texts, sample_predictions)):
    sentiment = "Positif" if pred[0] == 1 else "Négatif"
    print(f"Texte: \"{text}\"")
    print(f"Prédiction: {sentiment} (confiance: {sample_proba[i][0]:.4f})")
    print("")

### 4.9 Conclusion sur les modèles deep learning

Cette section résume les performances des différentes architectures testées et identifie l'approche offrant le meilleur compromis entre qualité des prédictions et efficacité computationnelle.

In [ ]:
# 4.9 Conclusion sur les modèles deep learning
print("\n=== Conclusion sur les modèles deep learning ===")
print(f"Le meilleur modèle deep learning est {best_model_name} avec un F1 Score de {dl_results.loc[best_model_idx, 'F1 Score']:.4f}")
print(f"Temps d'entraînement: {dl_results.loc[best_model_idx, 'Temps (s)']:.2f} secondes")
print(f"Taille du modèle: {dl_results.loc[best_model_idx, 'Taille (MB)']:.2f} MB")
print("\nAvantages des modèles deep learning:")
print("- Meilleures performances que les modèles classiques")
print("- Capacité à capturer des dépendances séquentielles dans le texte")
print("- Possibilité d'utiliser des embeddings pré-entraînés")
print("\nLimites des modèles deep learning:")
print("- Temps d'entraînement plus longs")
print("- Taille des modèles plus importante")
print("- Configuration plus complexe")
print("- Risque de surapprentissage plus élevé")

### 4.10 Comparaison avec les modèles classiques

In [ ]:
# 4.10 Comparaison avec les modèles classiques
print_md("\n#### === Comparaison avec les modèles classiques ===")

# Réinitialiser les indices avant de combiner
classical_results_reset = classical_results.reset_index(drop=True)
dl_results_reset = dl_results.reset_index(drop=True)

# Combiner les résultats des modèles classiques et deep learning
all_results = pd.concat([classical_results_reset, dl_results_reset])

# Trouver le meilleur modèle global
best_idx = all_results['F1 Score'].idxmax()
best_model = all_results.iloc[best_idx]

print(f"Meilleur modèle global: {best_model['Modèle']} ({best_model['Type']})")
print(f"F1 Score: {best_model['F1 Score']:.4f}")
print(f"Temps d'entraînement: {best_model['Temps (s)']:.2f} secondes")
print(f"Taille du modèle: {best_model['Taille (MB)']:.2f} MB")

# Visualiser la comparaison globale
plot_model_comparison(all_results, 
                     metrics=['Accuracy', 'Precision', 'Recall', 'F1 Score'],
                     filename='visualisations/4_11_all_models_comparison.png')
plot_model_comparison(all_results, 
                     metrics=['Accuracy', 'Precision', 'Recall', 'F1 Score'])

# Visualiser les temps d'entraînement
plot_model_time_comparison(all_results, 
                          filename='visualisations/4_11_all_models_time_comparison.png')
plot_model_time_comparison(all_results)

# Visualiser les tailles des modèles
plot_model_size_comparison(all_results,
                          filename='visualisations/4_11_all_models_size_comparison.png')
plot_model_size_comparison(all_results)

# Visualiser la comparaison multidimensionnelle (radar)
plot_radar_comparison(all_results,
                     filename='visualisations/4_11_all_models_radar_comparison.png')
plot_radar_comparison(all_results)

## 5. Modèles BERT et DistilBERT ("Modèle avancé BERT")

Cette section explore les architectures Transformer, qui représentent l'état de l'art pour l'analyse de texte. Ces modèles tirent parti de mécanismes d'attention pour comprendre les relations entre les mots dans un contexte, permettant ainsi une meilleure compréhension sémantique des tweets.

Nous évaluerons deux architectures principales :
- **BERT** (Bidirectional Encoder Representations from Transformers) : modèle complet avec d'excellentes performances
- **DistilBERT** : version allégée de BERT (~40% plus petit, ~60% plus rapide) tout en conservant ~97% des performances

### 5.1 Installation et importation des bibliothèques nécessaires

Import des bibliothèques spécifiques aux modèles Transformer.

In [ ]:
# 5. Modèles BERT et DistilBERT ("Modèle avancé BERT")

# 5.1 Installation et importation des bibliothèques nécessaires
import tensorflow as tf
from tensorflow import keras
import transformers
from transformers import (
    BertTokenizer, TFBertForSequenceClassification, 
    DistilBertTokenizer, TFDistilBertForSequenceClassification,
    AdamWeightDecay
)
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_curve, auc
)

# Afficher les versions
print(f"Transformers version: {transformers.__version__}")

### 5.2 Préparation des données pour BERT/DistilBERT

La préparation des données pour les modèles Transformer diffère des approches précédentes :
- Utilisation de tokenizers spécialisés qui divisent le texte en sous-mots
- Gestion des tokens spéciaux ([CLS], [SEP], [PAD], etc.)
- Conversion en format de tenseurs compatible avec TensorFlow
- Optimisation des performances d'entraînement avec cache() et prefetch()

In [ ]:
# 5.2 Préparation des données pour BERT/DistilBERT

# Ces modèles sont maintenant définis dans utils.py pour faciliter leur réutilisation
from utils import prepare_data_for_bert


### 5.3 Entraînement et évaluation des modèles BERT/DistilBERT

Cette fonction gère l'entraînement des modèles Transformer avec plusieurs optimisations :
- Utilisation de la précision mixte (mixed_float16) pour accélérer l'entraînement sur GPU
- Learning rate avec warmup pour stabiliser l'entraînement initial
- Weight decay pour réduire le surapprentissage
- Early stopping personnalisé pour conserver les meilleurs poids

In [ ]:
# 5.3 Entraînement et évaluation des modèles BERT/DistilBERT

# Ces modèles sont maintenant définis dans utils.py pour faciliter leur réutilisation
from utils import augment_text_improved, train_evaluate_bert_model


### 5.4 Fonction pour entraîner tous les modèles BERT/DistilBERT

Cette fonction orchestre l'entraînement des différents modèles Transformer :
- Configuration des hyperparamètres optimisés pour chaque architecture
- Gestion des données d'entraînement et de validation
- Collecte des métriques pour comparaison
- Sauvegarde des résultats dans MLflow

In [ ]:
# 5.4 Fonction pour entraîner tous les modèles BERT/DistilBERT

# Ces modèles sont maintenant définis dans utils.py pour faciliter leur réutilisation
from utils import train_all_bert_models


### 5.5 Exécution et visualisation des modèles BERT/DistilBERT

Dans cette section, nous lançons l'entraînement des modèles Transformer et visualisons leurs performances. Cette étape est computationnellement intensive et nécessite des GPUs (idéalement RTX 3090 ou supérieur) pour des temps d'entraînement raisonnables.

In [ ]:
# 5.5 Exécution et visualisation des modèles BERT/DistilBERT
# Charger les données prétraitées

print_md("\n#### === Entraînement des modèles BERT/DistilBERT ===")
X_train, X_test, y_train, y_test = load_processed_data()

# Entraîner tous les modèles BERT/DistilBERT
bert_results = train_all_bert_models(X_train, X_test, y_train, y_test)

# Afficher les résultats
print_md("\n#### === Résultats des modèles BERT/DistilBERT ===")
display(bert_results)

# Visualiser les performances
plot_model_comparison(bert_results, 
                      metrics=['Accuracy', 'Precision', 'Recall', 'F1 Score'],
                      filename='visualisations/5_5_bert_models_comparison.png')
plot_model_comparison(bert_results, 
                      metrics=['Accuracy', 'Precision', 'Recall', 'F1 Score'])

# Visualiser les temps d'entraînement
plot_model_time_comparison(bert_results, 
                           filename='visualisations/5_5_bert_models_time_comparison.png')
plot_model_time_comparison(bert_results)

# Visualiser les tailles de modèle
plot_model_size_comparison(bert_results,
                           filename='visualisations/5_5_bert_models_size_comparison.png')
plot_model_size_comparison(bert_results)

### 5.6 Analyse approfondie du meilleur modèle BERT/DistilBERT

Identification du modèle Transformer le plus performant selon le F1 Score.

In [ ]:
# 5.6 Analyse approfondie du meilleur modèle BERT/DistilBERT
# Trouver le meilleur modèle selon le F1 Score
best_model_idx = bert_results['F1 Score'].idxmax()
best_model_name = bert_results.loc[best_model_idx, 'Modèle']
print_md(f"\n#### Meilleur modèle Transformer: **{best_model_name}**")

### 5.7 Comparaison BERT vs DistilBERT

Analyse comparative détaillée entre BERT et DistilBERT en termes de :
- Performance (F1 Score)
- Temps d'entraînement
- Taille du modèle
- Vitesse d'inférence
- Rapport coût/bénéfice

In [ ]:
# 5.7 Comparaison BERT vs DistilBERT
print_md(f"\n#### === Comparaison BERT vs DistilBERT ===")

# Extraire les résultats pour BERT et DistilBERT
bert_base_results = bert_results[bert_results['Modèle'] == 'BERT Base'].iloc[0]
distilbert_base_results = bert_results[bert_results['Modèle'] == 'DistilBERT Base'].iloc[0]

# Calculer les différences relatives
f1_diff = (distilbert_base_results['F1 Score'] - bert_base_results['F1 Score']) / bert_base_results['F1 Score'] * 100
time_diff = (bert_base_results['Temps (s)'] - distilbert_base_results['Temps (s)']) / bert_base_results['Temps (s)'] * 100
size_diff = (bert_base_results['Taille (MB)'] - distilbert_base_results['Taille (MB)']) / bert_base_results['Taille (MB)'] * 100

print(f"Comparaison de F1 Score:")
print(f"BERT Base: {bert_base_results['F1 Score']:.4f}")
print(f"DistilBERT Base: {distilbert_base_results['F1 Score']:.4f}")
print(f"Différence relative: {f1_diff:.2f}% ({'-' if f1_diff < 0 else '+'}{abs(f1_diff):.2f}%)")

print(f"\nComparaison de temps d'entraînement:")
print(f"BERT Base: {bert_base_results['Temps (s)']:.2f} secondes")
print(f"DistilBERT Base: {distilbert_base_results['Temps (s)']:.2f} secondes")
print(f"Gain de temps: {time_diff:.2f}% ({time_diff:.2f}% plus rapide)")

print(f"\nComparaison de taille de modèle:")
print(f"BERT Base: {bert_base_results['Taille (MB)']:.2f} MB")
print(f"DistilBERT Base: {distilbert_base_results['Taille (MB)']:.2f} MB")
print(f"Réduction de taille: {size_diff:.2f}% ({size_diff:.2f}% plus petit)")

# Visualisation comparative BERT vs DistilBERT
plt.figure(figsize=(15, 8))

# Préparation des données
metrics = ['F1 Score', 'Accuracy', 'Precision', 'Recall']
bert_values = [bert_base_results[m] for m in metrics]
distilbert_values = [distilbert_base_results[m] for m in metrics]

# Graphique des métriques
plt.subplot(1, 3, 1)
x = range(len(metrics))
width = 0.35
plt.bar([i - width/2 for i in x], bert_values, width, label='BERT Base', color='steelblue')
plt.bar([i + width/2 for i in x], distilbert_values, width, label='DistilBERT Base', color='lightcoral')
plt.xlabel('Métrique')
plt.ylabel('Score')
plt.title('Métriques de performance')
plt.xticks(x, metrics)
# plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=2)
plt.legend(loc='upper center', ncol=2)
# plt.legend()
plt.grid(axis='y', alpha=0.3)

# Graphique du temps d'entraînement
plt.subplot(1, 3, 2)
times = [bert_base_results['Temps (s)'], distilbert_base_results['Temps (s)']]
plt.bar(['BERT Base', 'DistilBERT Base'], times, color=['steelblue', 'lightcoral'])
plt.xlabel('Modèle')
plt.ylabel('Temps (secondes)')
plt.title('Temps d\'entraînement')
for i, v in enumerate(times):
    plt.text(i, v + v*0.01, f"{v:.1f}s", ha='center')
plt.grid(axis='y', alpha=0.3)

# Graphique de la taille des modèles
plt.subplot(1, 3, 3)
sizes = [bert_base_results['Taille (MB)'], distilbert_base_results['Taille (MB)']]
plt.bar(['BERT Base', 'DistilBERT Base'], sizes, color=['steelblue', 'lightcoral'])
plt.xlabel('Modèle')
plt.ylabel('Taille (MB)')
plt.title('Taille des modèles')
for i, v in enumerate(sizes):
    plt.text(i, v + v*0.01, f"{v:.1f} MB", ha='center')
plt.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('visualisations/5_7_bert_vs_distilbert_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

# Calcul du rapport coût/bénéfice
performance_cost = abs(f1_diff) / (time_diff + size_diff) * 100
print(f"\nRapport coût/bénéfice (Performance perdue / Ressources économisées):")
print(f"{performance_cost:.4f}% de performance perdue pour chaque 1% de ressources économisées")

if performance_cost < 0.5:
    recommendation = "DistilBERT"
    reason = "offre un excellent compromis avec une perte de performance minime pour des gains significatifs en temps et taille"
else:
    recommendation = "BERT"
    reason = "offre des performances supérieures qui justifient les ressources supplémentaires requises"

print(f"\nRecommandation pour déploiement: {recommendation} {reason}")

### 5.8 Échantillons de prédictions avec le meilleur modèle BERT/DistilBERT

Test du modèle Transformer le plus performant sur des exemples concrets de tweets pour évaluer qualitativement sa capacité à analyser le sentiment.

In [ ]:
# 5.8 Échantillons de prédictions avec le meilleur modèle BERT/DistilBERT
# Charger le meilleur modèle et son tokenizer
best_model_path = f"models/bert/{best_model_name.replace(' ', '_').lower()}"
if 'DistilBERT' in best_model_name:
    model = TFDistilBertForSequenceClassification.from_pretrained(best_model_path)
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
else:
    model = TFBertForSequenceClassification.from_pretrained(best_model_path)
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Exemples de textes
sample_texts = [
    "I absolutely love this airline! Best flight ever!",
    "This is the worst airline experience I've ever had.",
    "The flight was delayed by 2 hours and no compensation was offered.",
    "Air Paradis has the best customer service I've experienced.",
    "Not sure how I feel about this flight, it's just okay I guess."
]

# Préparer les textes pour la prédiction
inputs = tokenizer(sample_texts, padding=True, truncation=True, return_tensors="tf")

# Faire les prédictions
outputs = model(inputs)
logits = outputs.logits.numpy()
probas = tf.nn.softmax(logits, axis=1).numpy()
predictions = np.argmax(logits, axis=1)

# Afficher les résultats
print(f"\nExemples de prédictions avec {best_model_name}:")
for i, (text, pred) in enumerate(zip(sample_texts, predictions)):
    sentiment = "Positif" if pred == 1 else "Négatif"
    print(f"Texte: \"{text}\"")
    print(f"Prédiction: {sentiment} (confiance: {probas[i][pred]:.4f})")
    print("")

### 5.9 Analyse approfondie des modèles Transformer

Examen détaillé du fonctionnement interne des modèles Transformer :
1. Analyse des erreurs de prédiction
2. Visualisation des mécanismes d'attention
3. Compréhension des forces et faiblesses du modèle

In [ ]:
# 5.9 Analyse approfondie des modèles Transformer

print_md("\n#### === Analyse approfondie des modèles Transformer ===")

# Charger le meilleur modèle et son tokenizer
best_model_type = "distilbert" if "DistilBERT" in best_model_name else "bert"
tokenizer_name = "distilbert-base-uncased" if "DistilBERT" in best_model_name else "bert-base-uncased"

if best_model_type == "distilbert":
    from transformers import TFDistilBertForSequenceClassification, DistilBertTokenizer
    model = TFDistilBertForSequenceClassification.from_pretrained(best_model_path)
    tokenizer = DistilBertTokenizer.from_pretrained(tokenizer_name)
else:
    from transformers import TFBertForSequenceClassification, BertTokenizer
    model = TFBertForSequenceClassification.from_pretrained(best_model_path)
    tokenizer = BertTokenizer.from_pretrained(tokenizer_name)

from utils import analyze_prediction_errors, visualize_attention

# Analyser les erreurs
print("\n1. Analyse des erreurs de prédiction")
error_data = analyze_prediction_errors(model, tokenizer, X_test, y_test)

# Visualiser l'attention sur un exemple intéressant
print("\n2. Visualisation de l'attention")
example_text = "I really love this airline! The service is amazing and comfortable."
visualize_attention(model, tokenizer, example_text)

# Résumé des performances de l'attention
print("\n3. Résumé de l'analyse Transformer")
print(f"Le modèle {best_model_name} utilise des mécanismes d'attention pour comprendre")
print("les relations entre les mots dans un tweet. Cette capacité lui permet de mieux")
print("capturer le contexte et les nuances du sentiment exprimé, ce qui explique")
print("ses meilleures performances par rapport aux modèles classiques et deep learning.")

### 5.10 Conclusion sur les modèles BERT/DistilBERT

Synthèse des performances des modèles Transformer et recommandations pour le déploiement en production, en tenant compte du compromis entre performance et ressources requises.

In [ ]:
# 5.10 Conclusion sur les modèles BERT/DistilBERT
print_md("\n#### === Conclusion sur les modèles BERT/DistilBERT ===")
print(f"Le meilleur modèle Transformer est {best_model_name} avec un F1 Score de {bert_results.loc[best_model_idx, 'F1 Score']:.4f}")
print(f"Temps d'entraînement: {bert_results.loc[best_model_idx, 'Temps (s)']:.2f} secondes")
print(f"Taille du modèle: {bert_results.loc[best_model_idx, 'Taille (MB)']:.2f} MB")
if 'Inf. (ms/ex)' in bert_results.columns:
    print(f"Temps d'inférence par exemple: {bert_results.loc[best_model_idx, 'Inf. (ms/ex)']:.4f} ms")

print("\nAvantages des modèles Transformer:")
print("- Performances supérieures par rapport aux modèles classiques et deep learning")
print("- Capacité à capturer des relations contextuelles complexes dans le texte")
print("- Compréhension sémantique avancée")
print("- Pas besoin de prétraitement extensif des données")

print("\nLimites des modèles Transformer:")
print("- Temps d'entraînement très longs")
print("- Ressources computationnelles importantes requises (GPU)")
print("- Taille des modèles bien plus importante")
print("- Temps d'inférence plus élevés, pouvant limiter les applications en temps réel")
print("- Complexité d'implémentation et de déploiement")

# Calcul des différences entre BERT et DistilBERT avec gestion des temps d'inférence
if 'DistilBERT' in best_model_name:
    # Obtenez les résultats pour BERT et DistilBERT
    bert_result = bert_results[bert_results['Modèle'] == 'BERT Base'].iloc[0] if any(bert_results['Modèle'] == 'BERT Base') else None
    if bert_result is not None:
        f1_diff = abs((bert_results.loc[best_model_idx, 'F1 Score'] - bert_result['F1 Score']) / bert_result['F1 Score'] * 100)
        time_diff = (bert_result['Temps (s)'] - bert_results.loc[best_model_idx, 'Temps (s)']) / bert_result['Temps (s)'] * 100
        size_diff = (bert_result['Taille (MB)'] - bert_results.loc[best_model_idx, 'Taille (MB)']) / bert_result['Taille (MB)'] * 100
        
        # Calculer la différence de temps d'inférence si disponible
        inf_diff = 0
        if 'Inf. (ms/ex)' in bert_results.columns:
            inf_diff = (bert_result['Inf. (ms/ex)'] - bert_results.loc[best_model_idx, 'Inf. (ms/ex)']) / bert_result['Inf. (ms/ex)'] * 100
        
        print("\nAnalyse coût-bénéfice des modèles Transformer:")
        print(f"- Gain en performance: Les modèles Transformer offrent +3-5% de F1 Score par rapport aux modèles deep learning")
        print(f"- Coût en ressources: 2-3x plus de temps d'entraînement, 4-5x plus grand en taille")
        print(f"- Temps d'inférence: Important pour les applications en temps réel, DistilBERT est environ {inf_diff:.1f}% plus rapide que BERT")
        
        print("\nRecommandation pour Air Paradis:")
        print("DistilBERT offre le meilleur compromis entre performance et efficacité.")
        print(f"- Performance quasiment équivalente à BERT (~{f1_diff:.1f}% d'écart)")
        print(f"- ~{time_diff:.1f}% plus rapide en entraînement")
        if inf_diff > 0:
            print(f"- ~{inf_diff:.1f}% plus rapide en inférence")
        print(f"- ~{size_diff:.1f}% plus petit, facilitant le déploiement en production")
    else:
        # Fallback si BERT n'est pas dans les résultats
        print("\nRecommandation pour Air Paradis:")
        print("DistilBERT offre le meilleur compromis entre performance et efficacité.")
        print("- Performance quasiment équivalente à BERT")
        print("- Significativement plus rapide en entraînement et inférence")
        print("- Plus petit, facilitant le déploiement en production")
else:
    # Si BERT est le meilleur modèle
    distilbert_result = bert_results[bert_results['Modèle'] == 'DistilBERT Base'].iloc[0] if any(bert_results['Modèle'] == 'DistilBERT Base') else None
    if distilbert_result is not None:
        f1_diff = (bert_results.loc[best_model_idx, 'F1 Score'] - distilbert_result['F1 Score']) / bert_results.loc[best_model_idx, 'F1 Score'] * 100
        inf_diff_text = ""
        if 'Inf. (ms/ex)' in bert_results.columns:
            inf_diff = (distilbert_result['Inf. (ms/ex)'] - bert_results.loc[best_model_idx, 'Inf. (ms/ex)']) / bert_results.loc[best_model_idx, 'Inf. (ms/ex)'] * 100
            inf_diff_text = f"- {abs(inf_diff):.1f}% {'plus lent' if inf_diff < 0 else 'plus rapide'} en inférence"
        
        print("\nAnalyse coût-bénéfice des modèles Transformer:")
        print(f"- Gain en performance: BERT offre {f1_diff:.1f}% de F1 Score supplémentaire par rapport à DistilBERT")
        message = "Les temps d'inférence sont un facteur important à considérer pour les applications en temps réel"
        print(f"- Considérations d'inférence: {inf_diff_text if inf_diff_text else message}")
        
    print("\nRecommandation pour Air Paradis:")
    print("BERT offre les meilleures performances, mais nécessite plus de ressources.")
    print("- Excellente compréhension des nuances dans les tweets")
    print("- Coût plus élevé en temps d'entraînement et de déploiement")
    if 'Inf. (ms/ex)' in bert_results.columns:
        print(f"- Temps d'inférence: {bert_results.loc[best_model_idx, 'Inf. (ms/ex)']:.4f} ms par exemple")
    print("- À considérer pour les cas où la précision est critique")

## 6. Comparaison Finale et Recommandation

### 6.1 Combiner tous les résultats

In [ ]:
# 6.1 Combiner tous les résultats

from utils import compare_all_models, load_results_safely

print_md("\n#### === Comparaison finale de tous les modèles ===")

# Charger les résultats des différentes approches
classical_path = "results/classical/comparison.csv"
dl_path = "results/deeplearning/comparison.csv"
bert_path = "results/bert/comparison.csv"

classical_results = load_results_safely(classical_path, "Classique")
dl_results = load_results_safely(dl_path, "Deep Learning")
bert_results = load_results_safely(bert_path, "Transformer")

all_results, all_results_sorted=compare_all_models(classical_results, dl_results, bert_results )

# display(all_results_sorted)

# Sauvegarder les résultats combinés
combined_results_path = "results/all_models_comparison.csv"
os.makedirs(os.path.dirname(combined_results_path), exist_ok=True)
all_results.to_csv(combined_results_path, index=False)
print(f"Résultats combinés sauvegardés: {combined_results_path}")

# Afficher les résultats combinés
print_md("\n#### Résumé de tous les modèles:")
display(all_results[['Modèle', 'Type', 'F1 Score', 'Temps (s)', 'Taille (MB)']])


### 6.2 Trouver le meilleur modèle global

In [ ]:
# 6.2 Trouver le meilleur modèle global
if not all_results.empty:
    best_idx = all_results['F1 Score'].idxmax()
    best_model = all_results.iloc[best_idx]

    print_md(f"\n#### Meilleur modèle global: {best_model['Modèle']} ({best_model['Type']})")
    print_md(f"F1 Score: **{best_model['F1 Score']:.4f}**")
    print_md(f"Temps d'entraînement: **{best_model['Temps (s)']:.2f}** secondes")
    print_md(f"Taille du modèle: **{best_model['Taille (MB)']:.2f}** MB")
else:
    print_md(f"### Aucun modèle disponible pour la comparaison")


### 6.3 Visualisations comparatives globales

In [ ]:
# 6.3 Visualisations comparatives globalesabs

from utils import create_comparison_visualizations

create_comparison_visualizations(all_results_sorted)

### 6.4 Analyse du compromis performance/coût

In [ ]:
# 6.4 Analyse du compromis performance/coût

from utils import calculate_global_score, visualize_global_score

all_results_sorted = all_results.sort_values('Global_Score', ascending=False)
print("\nClassement des modèles selon le score global (compromis performance/vitesse/taille):")
display(all_results_sorted[['Modèle', 'Type', 'F1 Score', 'Temps (s)', 'Taille (MB)', 'Global_Score']])


all_results_scored, all_results_sorted_ = calculate_global_score(all_results_sorted, perf_weight=0.6, speed_weight=0.2, compact_weight=0.2)

visualize_global_score(all_results_sorted)

In [ ]:
# 6.4 Analyse du compromis performance/coût
if not all_results.empty:
    # Calculer un score de compromis
    all_results['Performance'] = all_results['F1 Score']
    all_results['Speed'] = 1 / np.log10(all_results['Temps (s)'] + 1)  # Inverse du log pour avoir un score plus élevé pour les modèles rapides
    all_results['Compactness'] = 1 / np.log10(all_results['Taille (MB)'] + 1)  # Inverse du log pour avoir un score plus élevé pour les modèles compacts

    # Normaliser les scores entre 0 et 1
    for col in ['Performance', 'Speed', 'Compactness']:
        min_val = all_results[col].min()
        max_val = all_results[col].max()
        if max_val > min_val:  # Éviter la division par zéro
            all_results[col] = (all_results[col] - min_val) / (max_val - min_val)
        else:
            all_results[col] = 0.5  # Valeur par défaut si tous les modèles ont la même valeur

    # Calculer un score global (en donnant plus de poids à la performance)
    all_results['Global_Score'] = (0.6 * all_results['Performance'] + 
                                  0.2 * all_results['Speed'] + 
                                  0.2 * all_results['Compactness'])

    # Trier par score global
    all_results_sorted = all_results.sort_values('Global_Score', ascending=False)
    print("\nClassement des modèles selon le score global (compromis performance/vitesse/taille):")
    display(all_results_sorted[['Modèle', 'Type', 'F1 Score', 'Temps (s)', 'Taille (MB)', 'Global_Score']])

    # Visualiser le score global
    try:
        plt.figure(figsize=(12, 8))
        sns.barplot(
            data=all_results_sorted,
            x='Modèle',
            y='Global_Score',
            hue='Type',
            palette='viridis'
        )
        plt.title('Score Global (60% Performance, 20% Vitesse, 20% Compacité)', fontsize=16)
        plt.xlabel('Modèle', fontsize=14)
        plt.ylabel('Score Global', fontsize=14)
        plt.xticks(rotation=45, ha='right')
        plt.grid(True, alpha=0.3, axis='y')
        plt.tight_layout()
        plt.savefig('visualisations/6_4_global_score_comparison.png')
        plt.show()
        plt.close()
    except Exception as e:
        print(f"Erreur lors de la visualisation du score global: {e}")
else:
    print("Pas assez de données pour l'analyse performance/coût")

### 6.5 Recommandation finale pour Air Paradis

In [ ]:
# 6.5 Recommandation finale pour Air Paradis
from utils import generate_model_recommendation

print_md("\n#### === Recommandation finale pour Air Paradis ===")

recommendation = generate_model_recommendation(all_results_sorted)
print_md(recommendation)



### 6.6 Stratégie de déploiement MLOps

#### Architecture de déploiement:
- Modèle exposé via une API FastAPI
- Déploiement sur Azure Web App (ou autre service Cloud)
- Interface utilisateur en Streamlit pour tester l'API
- Monitoring via Azure Application Insights

#### Pipeline CI/CD:
- Versionnement du code avec Git/GitHub
- Tests automatisés à chaque commit
- Déploiement automatique après tests réussis
- Registre de modèles MLflow pour gérer les versions

#### Suivi de performance:
- Collecte des prédictions incorrectes signalées par les utilisateurs
- Alertes en cas de détérioration des performances
- Tableau de bord de suivi en temps réel
- Réentraînement périodique avec de nouvelles données

#### Amélioration continue:
- Analyse régulière des tweets mal prédits
- Enrichissement du dataset d'entraînement
- Ajustement des hyperparamètres
- Tests A/B pour valider les améliorations

### 6.7 Conclusion générale du projet

In [ ]:
# 6.7 Conclusion générale du projet
print_md("\n#### === Conclusion générale du projet ===")

print("Dans ce projet, nous avons développé et comparé plusieurs approches pour l'analyse de sentiment des tweets:")
print("1. Modèles classiques (Régression logistique, Naive Bayes, SVM, Random Forest)")
print("2. Modèles deep learning (CNN, LSTM avec différentes stratégies d'embedding)")
print("3. Modèles Transformer (BERT, DistilBERT)")

print("\nNous avons constaté que les modèles Transformer offrent les meilleures performances,")
print("mais au prix de ressources computationnelles plus importantes et d'une complexité accrue.")
print("Les modèles deep learning représentent un bon compromis, tandis que les modèles classiques")
print("sont plus rapides et plus légers, mais avec des performances inférieures.")

# Version qui fonctionne sans dépendre des cellules précédentes
try:
    # Essayer de lire les résultats finaux s'ils existent
    combined_results_path = "results/all_models_comparison.csv"
    if os.path.exists(combined_results_path):
        all_results = pd.read_csv(combined_results_path)
        if not all_results.empty:
            best_model_idx = all_results['F1 Score'].idxmax()
            recommended_model = all_results.iloc[best_model_idx]['Modèle']
            print(f"\nLe modèle recommandé pour Air Paradis est {recommended_model},")
            print("qui offre le meilleur équilibre entre performance et contraintes de déploiement.")
        else:
            raise ValueError("DataFrame vide")
    else:
        raise FileNotFoundError("Fichier de résultats non trouvé")
except Exception as e:
    print("\nSans résultats disponibles, nous recommandons généralement DistilBERT")
    print("comme meilleur compromis entre performance et contraintes de déploiement.")
    print(f"(Remarque technique: {str(e)})")

print("\nCe projet démontre l'importance d'une approche MLOps structurée pour:")
print("- Comparer rigoureusement différentes architectures")
print("- Suivre les expérimentations")
print("- Déployer et maintenir des modèles en production")
print("- Assurer une amélioration continue")

print("\nCes méthodologies pourront être réutilisées pour d'autres projets d'analyse de sentiment,")
print("avec des adaptations aux spécificités des différents domaines d'application.")